In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
from datasets import Dataset
classes = ["banking","valuation","household","real estate","corporate","external","sovereign","technology", "climate", "energy", "health", "eu"]


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,top_k_accuracy_score
import math
import pickle

In [4]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(classes))
finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert', use_fast =True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
# object_methods = [method_name for method_name in dir(finbert)
#                   if callable(getattr(finbert, method_name))]
# print(object_methods)
finbert_weights = finbert.state_dict()
model_weights = model.state_dict()
del finbert_weights["bert.pooler.dense.weight"]
del finbert_weights["bert.pooler.dense.bias"]
del finbert_weights["classifier.weight"]
del finbert_weights["classifier.bias"]
finbert_weights["bert.pooler.dense.weight"] = model_weights["bert.pooler.dense.weight"]
finbert_weights["bert.pooler.dense.bias"] = model_weights["bert.pooler.dense.bias"]
finbert_weights["classifier.weight"] = model_weights["classifier.weight"]
finbert_weights["classifier.bias"] = model_weights["classifier.bias"]

model.load_state_dict(finbert_weights)


<All keys matched successfully>

In [6]:
# flatten to one list for all 3
with open('train_data.pickle', 'rb') as file:
    train = pickle.load(file)

with open('gpt.pickle', 'rb') as file:
    gpt = pickle.load(file)

with open('gpt_p2.pickle', 'rb') as file:
    gpt2 = pickle.load(file)
    
gpt = [item for sublist in gpt for item in sublist]
gpt2 = [item for sublist in gpt2 for item in sublist]

mixed = gpt + gpt2

In [27]:
print(len(mixed))

2458


In [28]:
sample = 1
# text = [item["text"] for i in range(sample) for item in mixed]
# label = [np.random.choice(len(classes), p=item["dist"]) for i in range(sample) for item in mixed]
additional_text = []
additional_label = []

for idx in range(1,len(mixed)-1):
    sent1, sent2 = mixed[idx-1:idx+1]
    for i in range(1):
        additional_text.append(sent1["text"]+' '+ sent2["text"])
        dist = (np.array(sent1["dist"]) + np.array(sent2["dist"]))/2
        additional_label.append(np.argmax(dist))

for idx in range(2,len(mixed)-1):
    sent1, sent2, sent3 = mixed[idx-2:idx+1]
    for i in range(1):
        additional_text.append(sent1["text"]+' '+ sent2["text"]+' '+sent3["text"])
        dist = (np.array(sent1["dist"]) + np.array(sent2["dist"])+ np.array(sent3["dist"]))/3
        additional_label.append(np.argmax(dist))

text_max = [item["text"] for i in range(sample) for item in train]
label_max = [np.argmax(item["dist"]) for i in range(sample) for item in train]

text_max_mixed = [item["text"] for i in range(sample) for item in mixed]
label_max_mixed = [np.argmax(item["dist"]) for i in range(sample) for item in mixed]

# comb_text = text_max+text_max_mixed + additional_text
# comb_label = label_max+label_max_mixed + additional_label

In [29]:
print(len(text_max))

261


In [32]:
import random
def randomize_split(text, label):
    temp = list(zip(text, label))
    random.shuffle(temp)
    comb_text, comb_label =  zip(*temp)
    return comb_text, comb_label

a1 = randomize_split(text_max,label_max)
a2 = randomize_split(text_max_mixed,label_max_mixed)
a3 = randomize_split(additional_text,additional_label)
# comb_text = 
# comb_label = 
train_text, test_text =  [*a1[0][math.ceil(len(a1)*0.8):],*a2[0][math.ceil(len(a2)*0.8):],*a3[0][math.ceil(len(a2)*0.8):] ], [*a1[0][:math.ceil(len(a1)*0.8)],*a2[0][:math.ceil(len(a2)*0.8)],*a3[0][:math.ceil(len(a2)*0.8)] ]
train_label, test_label = [*a1[1][math.ceil(len(a1)*0.8):],*a2[1][math.ceil(len(a2)*0.8):],*a3[1][math.ceil(len(a2)*0.8):] ],  [*a1[1][:math.ceil(len(a1)*0.8)],*a2[1][:math.ceil(len(a2)*0.8)],*a3[1][:math.ceil(len(a2)*0.8)] ]
# train_text, test_text = comb_text[:math.ceil(len(comb_text)*0.8)], comb_text[math.ceil(len(comb_text)*0.8):]
# train_label, test_label = comb_label[:math.ceil(len(comb_label)*0.8)], comb_label[math.ceil(len(comb_text)*0.8):]

261
2458
4911


In [50]:
print(len([*a1[0][math.ceil(len(a1)*0.8):],*a2[0][math.ceil(len(a2)*0.8):],*a3[0][math.ceil(len(a2)*0.8):] ]))
print(len(a1[0][math.ceil(len(a2)*0.8):]))
print(len(a2[0][math.ceil(len(a2)*0.8):]))
print(len(test_label))

7624
259
2456
7624


In [ ]:

# with open('dataset_57.pickle', 'wb') as file:
#     pickle.dump((comb_text,comb_label), file)

In [ ]:

# with open('dataset_57.pickle', 'rb') as file:
#     comb_text,comb_label = pickle.load(file)

In [13]:
# assert len(comb_text) == len(comb_label)

NameError: name 'comb_text' is not defined

In [ ]:
# print(len(comb_label))

7630


In [14]:
train_dataset = Dataset.from_dict({"text":train_text, "label":train_label})
test_dataset = Dataset.from_dict({"text":test_text, "label":test_label})

In [15]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

100%|██████████| 1/1 [00:01<00:00,  1.39s/ba]


In [16]:
train_dataset["input_ids"]

tensor([[  101,  1996,  1057,  1012,  1055,  1012,  6451,  4753,  1010,  1999,
          2029,  5860, 11795,  3085,  3431,  1999,  1996,  2943,  4666,  4050,
          5258,  3254,  1010,  5281,  6937,  1998, 27885,  8043, 12423,  3431,
          2058,  1996,  2627,  2809,  2086,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  1996,  2117,  4403,  1997,  1996,  5325,  2234,  2004,  1037,
          9509,  1997,  1996,  3279,  1997,  7023,  1999,  2070, 11074,  2015,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0

In [17]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    top3 = top_k_accuracy_score(labels, pred.predictions,k=3)
    top2 = top_k_accuracy_score(labels, pred.predictions,k=2)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'top3': top3,
         'top2': top2
    }

training_args = TrainingArguments(
    
    output_dir='./results',
    learning_rate=2e-5,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    evaluation_strategy='epoch',
    save_strategy = "epoch",
    logging_dir='./logs',
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
)
    # "test-glue",
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    # learning_rate=2e-5,
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    # weight_decay=0.01,
    # load_best_model_at_end=True,
    # metric_for_best_model=metric_name,
    # push_to_hub=True,
    # push_to_hub_model_id=f"{model_name}-finetuned-{task}",
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [18]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 6
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20
  5%|▌         | 1/20 [00:02<00:40,  2.15s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 7624
  Batch size = 64
C:\Users\abc\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

{'eval_loss': 2.624227285385132, 'eval_accuracy': 0.07568205666316893, 'eval_f1': 0.03992944643621367, 'eval_precision': 0.05949892829971521, 'eval_recall': 0.07700902060399635, 'eval_top3': 0.21497901364113325, 'eval_top2': 0.14034627492130117, 'eval_runtime': 21.0079, 'eval_samples_per_second': 362.912, 'eval_steps_per_second': 5.712, 'epoch': 1.0}


Model weights saved in ./results\checkpoint-1\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-7258] due to args.save_total_limit
 10%|█         | 2/20 [00:46<08:01, 26.78s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 7624
  Batch size = 64
C:\Users\abc\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 10%|█         | 2/20 [01:05<08:01, 26.78s/it]Saving model checkpoint to ./results\checkpoint-2
Configuration saved in ./results\checkpoint-2\config.json


{'eval_loss': 2.62414288520813, 'eval_accuracy': 0.07581322140608604, 'eval_f1': 0.039978494851632386, 'eval_precision': 0.05955901404612052, 'eval_recall': 0.07704647378751696, 'eval_top3': 0.21497901364113325, 'eval_top2': 0.14060860440713535, 'eval_runtime': 19.6475, 'eval_samples_per_second': 388.039, 'eval_steps_per_second': 6.108, 'epoch': 2.0}


Model weights saved in ./results\checkpoint-2\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-7640] due to args.save_total_limit
 15%|█▌        | 3/20 [01:28<09:34, 33.78s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 7624
  Batch size = 64
C:\Users\abc\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 15%|█▌        | 3/20 [01:48<09:34, 33.78s/it]Saving model checkpoint to ./results\checkpoint-3
Configuration saved in ./results\checkpoint-3\config.json


{'eval_loss': 2.623938798904419, 'eval_accuracy': 0.07633788037775446, 'eval_f1': 0.040214954491218834, 'eval_precision': 0.059933722621652384, 'eval_recall': 0.07719628652159936, 'eval_top3': 0.21511017838405036, 'eval_top2': 0.141395592864638, 'eval_runtime': 20.3626, 'eval_samples_per_second': 374.412, 'eval_steps_per_second': 5.893, 'epoch': 3.0}


Model weights saved in ./results\checkpoint-3\pytorch_model.bin


RuntimeError: [enforce fail at ..\caffe2\serialize\inline_container.cc:298] . unexpected pos 215880064 vs 215879952

In [ ]:
trainer.evaluate()
# "raw"
# {'eval_loss': 0.003656767075881362,
#  'eval_accuracy': 0.9995957145744896,
#  'eval_f1': 0.9996459266546559,
#  'eval_precision': 0.9997773468361704,
#  'eval_recall': 0.999515747499324,
#  'eval_top3': 0.9997978572872448,
#  'eval_top2': 0.9997978572872448,
#  'eval_runtime': 7.5519,
#  'eval_samples_per_second': 655.065,
#  'epoch': 2.02}

# "enhancement"
# {'eval_loss': 1.0243563652038574,
#  'eval_accuracy': 0.5766404864454016,
#  'eval_f1': 0.5316538589564456,
#  'eval_precision': 0.546277783485254,
#  'eval_recall': 0.5233829825328863,
#  'eval_top3': 0.8826957182670383,
#  'eval_top2': 0.7907271345325564,
#  'eval_runtime': 9.3876,
#  'eval_samples_per_second': 420.447,
#  'epoch': 20.0}

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1526
  Batch size = 64
100%|██████████| 24/24 [00:03<00:00,  6.37it/s]


{'eval_loss': 1.3667666912078857,
 'eval_accuracy': 0.8289646133682831,
 'eval_f1': 0.7600767747856746,
 'eval_precision': 0.7783009182422279,
 'eval_recall': 0.74921395397379,
 'eval_top3': 0.936435124508519,
 'eval_top2': 0.9121887287024901,
 'eval_runtime': 3.9514,
 'eval_samples_per_second': 386.188,
 'eval_steps_per_second': 6.074,
 'epoch': 20.0}

In [ ]:

predict_dataset = Dataset.from_dict({"text":["In contrast to the radical forces buffeting valuations, for most companies, 2020 was a year of “strategy lockdown.",
"Domestic policies thus tended to reinforce negative spillovers and exacerbate systemic risk across the euro area.",
"Mortgage interest rate in selected European countries as of 4th quarter of 2019 and 2020 increased",
"Accordingly, I shall spend most of my allotted time outlining the chosen monetary policy instruments and procedures of the ESCB and the considerations that have been raised relating to strategy.",
"Credit card interest rate has gone up a lot."]})

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
predict_dataset = predict_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))

trainer.predict(predict_dataset)

100%|██████████| 1/1 [00:00<00:00, 167.14ba/s]
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 5
  Batch size = 64
  0%|          | 0/1 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.54258686, -2.283698  , -1.2337625 ,  0.6101017 ,  8.834227  ,
        -0.60477626, -0.1470656 , -0.25806686, -1.9158627 , -0.45982993,
        -0.08515993, -1.3472333 ],
       [ 9.702999  , -0.29276183, -1.5581697 , -1.5263258 , -1.2451688 ,
        -0.5294447 ,  2.195682  , -1.8384721 , -1.5489938 , -3.0295794 ,
        -1.7577528 , -1.730811  ],
       [ 1.4455788 , -1.5327429 ,  2.143541  ,  7.9250283 , -1.1390123 ,
        -3.0250149 , -0.9481666 , -2.0208018 ,  0.02337106, -0.8389467 ,
        -0.08845273, -0.9033677 ],
       [-1.1007565 , -1.3134553 , -1.9952139 , -0.77075744, -1.4220358 ,
         0.21309426, 10.992527  , -0.8264481 , -0.6472416 , -2.5305545 ,
        -1.0293103 , -1.4311087 ],
       [ 2.7709064 ,  7.787258  ,  2.5337188 , -1.0464295 , -2.2166548 ,
        -1.2778659 , -2.6960404 , -2.199739  , -1.8598685 , -1.4836167 ,
        -1.3202602 , -1.9072998 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 0.0389, 

In [ ]:
classes = ["banking","valuation","household","real estate","corporate","external","sovereign","technology", "climate", "energy", "health", "eu"]
